# 📊 FAW Binary Classification — Metrics & Visualizations (by Ridor)This notebook evaluates the trained binary model and visualizes its performance using various metrics.

In [ ]:
import torchfrom torchvision import datasets, transformsfrom torch.utils.data import DataLoaderimport matplotlib.pyplot as pltimport seaborn as snsimport numpy as npfrom sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_curve, auc

In [ ]:
# 🔧 Load test datasetdata_dir = '/content/FAW-AI-BINARY-MODEL/dataset_split/test'transform = transforms.Compose([    transforms.Resize((224, 224)),    transforms.ToTensor(),    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])test_dataset = datasets.ImageFolder(data_dir, transform=transform)test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)print(f'Test Samples: {len(test_dataset)} | Classes: {test_dataset.classes}')

In [ ]:
# ⚙️ Load modelfrom torchvision import modelsimport torch.nn as nndevice = torch.device('cuda' if torch.cuda.is_available() else 'cpu')model = models.resnet18(weights=None)model.fc = nn.Linear(model.fc.in_features, 2)  # Binary classificationmodel.load_state_dict(torch.load('/content/FAW-AI-BINARY-MODEL/best_faw_binary.pth', map_location=device))model.to(device)model.eval()print('✅ Model loaded successfully!')

In [ ]:
# 🧮 Generate predictionsall_labels, all_preds, all_probs = [], [], []with torch.no_grad():    for images, labels in test_loader:        images, labels = images.to(device), labels.to(device)        outputs = model(images)        probs = torch.softmax(outputs, dim=1)[:, 1]        _, preds = torch.max(outputs, 1)        all_labels.extend(labels.cpu().numpy())        all_preds.extend(preds.cpu().numpy())        all_probs.extend(probs.cpu().numpy())print('✅ Predictions complete.')

In [ ]:
# 📈 Compute metricsacc = accuracy_score(all_labels, all_preds)prec = precision_score(all_labels, all_preds)rec = recall_score(all_labels, all_preds)f1 = f1_score(all_labels, all_preds)print(f'Accuracy: {acc:.3f}')print(f'Precision: {prec:.3f}')print(f'Recall: {rec:.3f}')print(f'F1 Score: {f1:.3f}')

In [ ]:
# 🧾 Confusion Matrixcm = confusion_matrix(all_labels, all_preds)sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=test_dataset.classes, yticklabels=test_dataset.classes)plt.xlabel('Predicted')plt.ylabel('Actual')plt.title('Confusion Matrix — FAW Binary Model')plt.show()

In [ ]:
# 🧭 ROC Curvefpr, tpr, _ = roc_curve(all_labels, all_probs)roc_auc = auc(fpr, tpr)plt.plot(fpr, tpr, color='blue', label=f'AUC = {roc_auc:.3f}')plt.plot([0, 1], [0, 1], 'r--')plt.xlabel('False Positive Rate')plt.ylabel('True Positive Rate')plt.title('ROC Curve')plt.legend()plt.show()